In [1]:
import pandas as pd
import numpy as np
import calmap
import os
import re
import json as JSON
PATH=''

def checkDirectory():
    if os.path.isdir('./jsons')==False:
        !mkdir jsons    

def initialise():
    checkDirectory()
    return './datasets/'

    
PATH=initialise()

In [2]:
def load_match_data(filenames):
    match_regex = re.compile(r'^(?:.*)_match.csv$')
    matches = [m for m in map(match_regex.match, filenames) if m is not None]
    for match in matches:
        match_type = re.sub('\_match.csv$', '', match.group(0))
        print("Match:"+match_type)
        dataset = pd.read_csv(PATH+match.group(0))
        dataset.insert(2, "type", match_type)
        yield dataset

def load_innings_data(filenames):
    inning_regex = re.compile(r'^(?:.*)_innings.csv$')
    innings = [m for m in map(inning_regex.match, filenames) if m is not None]
    for inning in innings:
        inning_type = re.sub('\_innings.csv$', '', inning.group(0))
        print("Inning:"+inning_type)
        dataset = pd.read_csv(PATH+inning.group(0))
        dataset.insert(2, "type", inning_type)
        yield dataset

data_match=pd.concat(load_match_data(os.listdir(PATH)))
data_innings=pd.concat(load_innings_data(os.listdir(PATH)))
print(data_match.size)
print(data_innings.size)

Match:odi
Match:t20
Match:test
Inning:odi


C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:22: DtypeWarning: Columns (1,2,3,6,7,8,9,17,18,20,21,22,26,27) have mixed types.Specify dtype option on import or set low_memory=False.


Inning:t20
Inning:test


C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:22: DtypeWarning: Columns (1,2,3,6,7,8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.


177096
11702428


In [3]:
merged_dataset = pd.merge(data_innings,data_match,on=['Date','Ground'])

In [4]:
def generateName(string1="",string2="",string3=""):
    string=string1+'_'+string2+'_'+string3
    string=re.sub('-+', '_', string)
    string=re.sub('\s+', '_', string)
    return string
def writeToFile(jsonObject,name):
    checkDirectory()
    with open('./jsons/'+name+'.json', 'w') as outfile:
        JSON.dump(jsonObject, outfile)

In [5]:
def removeUnnecessaryCountryData(dataset):
    countries=['Afghanistan','Zimbabwe']
    for country in countries:
        dataset=dataset[dataset['Country_x']!=country]
        dataset=dataset[dataset['Country_y']!=country]        
    return dataset
def updateCommentedCode(df):
    df=df[df['Match Year']==2017]
    df=df[df['Match Month']=='Jan']
    dates=df['Date'].unique()  
    matches=df['Match'].unique()  
    date=dates[0]
    match=matches[0]
    df=df[df['Date']==date]
    df=df[df['Match']==match]
    df=df[df['Innings Number']=='2']
    df
    df=df.sort_values('Innings Number',ascending=True)
    return df
    
def generateDataOfMatch(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df=df[["Innings Player","Innings Runs Scored Num","type_x","Innings Batted Flag","Innings Batting Strike Rate","Innings Number","Opposition","Ground","Date","Country_x","Country_y","Innings Overs Bowled","Innings Bowled Flag","Innings Runs Conceded","Innings Wickets Taken","Innings Economy Rate","Result","Margin","Match","Home/Away","Match Month","Match Year","Match Period"]]
    df=removeUnnecessaryCountryData(df)
    df=df[df['Home/Away']=='Home']
    df.fillna(0,inplace=True)
    df['Innings Runs Scored Num'].fillna(0,inplace=True)
    df['Innings Runs Scored Num'].replace(to_replace ="-", value =0,inplace=True)
#     df.to_csv('hjk.csv')
    match_array=[]
    for key,group_df in df.groupby(['Match','Date','Ground','Result','Margin','type_x']):
            json={}
            json['match']=key[0]
            teams=key[0].split(' v ')
            json['team_1']=teams[0]
            json['team_2']=teams[1]
            json['date']=key[1]
            json['ground']=key[2]
            json['type']=key[5]
            innings_array=[]
            group_df = group_df.sort_values(by='Innings Number', ascending=True)
            for subkey,subgroup_df in group_df.groupby(['Innings Number']):
                innings={}
                innings['number']=subkey[0]
                player_array=[]
                for index, player_individual in subgroup_df.iterrows():
                    player={}
                    batting={}
                    bowling={}
                    player['name']=player_individual['Innings Player']
                    player['country']=player_individual['Country_x']
                    batting['runs']=player_individual['Innings Runs Scored Num']
                    bowling['runs_given']=player_individual['Innings Runs Conceded']
                    bowling['economy_rate']=player_individual['Innings Economy Rate']                
                    bowling['over_bowled']=player_individual['Innings Overs Bowled']
                    bowling['wicket']=player_individual['Innings Wickets Taken']
                    batting['batted']=True if (player_individual['Innings Batted Flag']==1) else False
                    bowling['bowled']=True if (player_individual['Innings Bowled Flag']==1) else False
                    batting['strike_rate']=player_individual['Innings Batting Strike Rate']
                    player['batting']=batting
                    player['bowling']=bowling
                    player['innings']=player_individual['Innings Number']
                    player_array.append(player)
                innings['players']=player_array
                innings_array.append(innings)                
            json['innings']=innings_array
            writeToFile(json,generateName(key[0],key[1],key[2]))
            match_array.append(json)

In [6]:
generateDataOfMatch(merged_dataset)